# Using the HyP3 SDK for Python

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jiangyuinsar/hyp3-sdk/develop?filepath=docs%2Fsdk_example_2020MonteCristo.ipynb)

HyP3's Python SDK `hyp3_sdk` provides a convenience wrapper around the HyP3 API and HyP3 jobs.


The HyP3 SDK can be installed using [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
 (recommended) via [`conda`](https://anaconda.org/conda-forge/hyp3_sdk):

```
conda install -c conda-forge hyp3_sdk
```

Or using [`pip`](https://pypi.org/project/hyp3-sdk/):

```
python -m pip install hyp3_sdk
```

Full documentation of the SDK can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [ ]:
# initial setup
!pip install hyp3_sdk
import hyp3_sdk as sdk
from hyp3_sdk import asf_search

## Authenticating to the API

The SDK will attempt to pull your [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) credentials out of `~/.netrc`
by default, or you can pass your credentials in directly

In [ ]:
# enter your credentials
hyp3 = sdk.HyP3(prompt=True)

## Submitting jobs

The SDK provides a submit method for [all supported job types](https://hyp3-docs.asf.alaska.edu/products/).

### Submitting Sentinel-1 InSAR jobs

The SDK can also submit Sentinel-1 Interferometric Synthetic Aperture Radar (InSAR) jobs. Using the example granule list
for our RTC jobs as the reference scenes, we can find their nearest and next-nearest neighbor granules, and submit them
as pairs for InSAR processing.


In [ ]:
pairs = [('S1A_IW_SLC__1SDV_20200429T135117_20200429T135144_032343_03BE57_E7C0', 'S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768'),
         ('S1A_IW_SLC__1SDV_20200429T135117_20200429T135144_032343_03BE57_E7C0', 'S1A_IW_SLC__1SDV_20200523T135118_20200523T135145_032693_03C96E_C56D'),
         ('S1A_IW_SLC__1SDV_20200429T135117_20200429T135144_032343_03BE57_E7C0', 'S1A_IW_SLC__1SDV_20200604T135118_20200604T135145_032868_03CE9E_1538'), 
         ('S1A_IW_SLC__1SDV_20200429T135117_20200429T135144_032343_03BE57_E7C0', 'S1A_IW_SLC__1SDV_20200616T135119_20200616T135146_033043_03D3DF_94E7'), 
         ('S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768', 'S1A_IW_SLC__1SDV_20200523T135118_20200523T135145_032693_03C96E_C56D'), 
         ('S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768', 'S1A_IW_SLC__1SDV_20200604T135118_20200604T135145_032868_03CE9E_1538'), 
         ('S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768', 'S1A_IW_SLC__1SDV_20200616T135119_20200616T135146_033043_03D3DF_94E7'), 
         ('S1A_IW_SLC__1SDV_20200523T135118_20200523T135145_032693_03C96E_C56D', 'S1A_IW_SLC__1SDV_20200604T135118_20200604T135145_032868_03CE9E_1538'), 
         ('S1A_IW_SLC__1SDV_20200523T135118_20200523T135145_032693_03C96E_C56D', 'S1A_IW_SLC__1SDV_20200616T135119_20200616T135146_033043_03D3DF_94E7'), 
         ('S1A_IW_SLC__1SDV_20200604T135118_20200604T135145_032868_03CE9E_1538', 'S1A_IW_SLC__1SDV_20200616T135119_20200616T135146_033043_03D3DF_94E7')]

insar_jobs = []
place_energetic_event = place_for_asf_max_events[event_id_energetic_event]
project_name = '2020 Monte Cristo Range'
for pair in pairs:
    reference_granule = pair[0]
    secondary_granule = pair[1]
    insar_jobs += hyp3.submit_insar_job(reference_granule, secondary_granule,
                                        name=project_name, include_look_vectors=True,
                                        include_dem=True)
print(insar_jobs)

AutoRIFT does not currently accept any keyword arguments for product customization.

## Monitoring jobs

One jobs are submitted, you can either watch the jobs until they finish

which will require you to keep the cell/terminal running, or you can come back later and search for jobs

In [ ]:
batch = hyp3.find_jobs(name=project_name)

if batch.complete():
    #filter to only succeeded jobs
    succeeded_jobs = batch.filter_jobs(succeeded=True, running=False, failed=False)

    #download files if not downloaded already
    for job in succeeded_jobs.jobs:
        filename = job.to_dict()['files'][0]['filename']
        location = os.path.join(project_name,filename)
        if not os.path.exists(location):
            job.download_files(location=project_name,create=True)
        
    # #download files if not downloaded already
    # file_list = succeeded_jobs.download_files(location=project_name,create=True)
else:
    #to get updated information
    batch = hyp3.refresh(batch)
    #or to wait until completion and get updated information (which will take a fair bit)
    batch = hyp3.watch(batch)

### Downloading files

Batches are collections of jobs. They provide a snapshot of the job status when the job was created or last
refreshed. To get updated information on a batch

In [ ]:
print(insar_jobs)
insar_jobs = hyp3.refresh(insar_jobs)
print(insar_jobs)

`hyp3.watch()` will return a refreshed batch once every job in the batch has completed.

Batches can be added together

In [ ]:
print(f'Number of Jobs:\n  RTC:{len(rtc_jobs)}\n  InSAR:{len(insar_jobs)}\n  autoRIFT:{len(autorift_jobs)}')
all_jobs = rtc_jobs + insar_jobs + autorift_jobs
print(f'Total number of Jobs: {len(all_jobs)}')

You can check the status of a batch (at last refresh) by printing the batch

In [ ]:
print(all_jobs)

and filter jobs by status

In [ ]:
succeeded_jobs = all_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = all_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

You can download the files for all successful jobs

In [ ]:
file_list = succeeded_jobs.download_files()

*Note: only succeeded jobs will have files to download.*